# Optimiser Interface
## Interacting with optimisers in PyBOP

This notebook demonstrates how to interact with PyBOP's optimiser classes.

### Setting up the Environment

If you don't already have PyBOP installed, check out the [installation guide](https://pybop-docs.readthedocs.io/en/latest/installation.html) first.

We begin by importing the necessary libraries. Let's also fix the random seed to generate consistent output during development.

In [ ]:
import json

import numpy as np
import pybamm

import pybop

pybop.plot.PlotlyManager().pio.renderers.default = "notebook_connected"

np.random.seed(8)  # users can remove this line

## Setting up the problem

The code block below sets up the model, parameter, dataset and creates the problem. For more information on this process, take a look at other notebooks in the examples directory.

In [ ]:
# Load the parameters
json_path = "../../parameters/initial_ecm_parameters.json"
with open(json_path) as f:
    params_dict = json.load(f)

param = pybamm.ParameterValues(params_dict)
param.update(
    {
        "Open-circuit voltage [V]": pybamm.equivalent_circuit.Thevenin().default_parameter_values[
            "Open-circuit voltage [V]"
        ]
    },
    check_already_exists=False,
)

# Define the model
model = pybamm.equivalent_circuit.Thevenin(options={"number of rc elements": 1})

# Define the parameters
parameters = [
    pybop.Parameter(
        "R0 [Ohm]",
        prior=pybop.Gaussian(0.0002, 0.0001),
        bounds=[1e-4, 1e-2],
    )
]

# Generate synthetic data
t_eval = np.arange(0, 900, 2)
sim = pybamm.Simulation(model, parameter_values=parameter_values)
sol = sim.solve(t_eval=t_eval)

# Form dataset
dataset = pybop.Dataset(
    {
        "Time [s]": t_eval,
        "Current function [A]": sol["Current [A]"](t_eval),
        "Voltage [V]": sol["Voltage [V]"](t_eval),
    }
)

# Construct builder
builder = (
    pybop.builders.Pybamm()
    .set_dataset(dataset)
    .set_simulation(model, parameter_values=parameter_values)
    .add_cost(pybop.costs.pybamm.SumSquaredError("Voltage [V]", "Voltage [V]"))
)
for param in parameters:
    builder.add_parameter(param)

problem = builder.build()

## Interacting with the Optimisers

Now that we have set up the required objects, we can introduce the interface for interacting with the optimisers. In this example, we will see how to use PINTS optimisers and SciPy optimisers. Here is the complete [list of optimisers](https://github.com/pybop-team/PyBOP?tab=readme-ov-file#supported-methods) supported in PyBOP.

In the following example, PINTS-based optimiser set-up is shown. Optimiser arguments can be passed using `PintsOptions` class.

In [ ]:
options = pybop.PintsOptions(
    max_unchanged_iterations=20,
    max_iterations=50,
)
optim_one = pybop.PSO(problem, options=options)
results = optim_one.run()

Next, the use of SciPy optimisers is demonstrated. `ScipyMinimizeOptions` class is used to pass the optimiser arguments.

In [ ]:
options = pybop.ScipyMinimizeOptions(
    maxiter=50,
    method="Nelder-Mead",
    tol=1e-9,
)
optim_two = pybop.SciPyMinimize(problem, options=options)
results2 = optim_two.run()

Iter: 1 | Evals: 4 | Best Values: [0.00024049] | Best Cost: 0.0064896983490682625 |
Iter: 2 | Evals: 6 | Best Values: [0.00028231] | Best Cost: 0.005794642453313131 |
Iter: 3 | Evals: 8 | Best Values: [0.00036596] | Best Cost: 0.004522605365454388 |
Iter: 4 | Evals: 10 | Best Values: [0.00053326] | Best Cost: 0.002450830004345302 |
Iter: 5 | Evals: 12 | Best Values: [0.00086785] | Best Cost: 0.0001964745404737031 |
Iter: 6 | Evals: 14 | Best Values: [0.00103515] | Best Cost: 1.3894437581354766e-05 |
Iter: 7 | Evals: 16 | Best Values: [0.00103515] | Best Cost: 1.3894437581354766e-05 |
Iter: 8 | Evals: 18 | Best Values: [0.00099332] | Best Cost: 5.021114904837217e-07 |
Iter: 9 | Evals: 20 | Best Values: [0.00099332] | Best Cost: 5.021114904837217e-07 |
Iter: 10 | Evals: 22 | Best Values: [0.00100378] | Best Cost: 1.603585248541418e-07 |
OptimisationResult:
  Best result from 1 run(s).
  Initial parameters: [0.00020912]
  Optimised parameters: [0.001]
  Diagonal Fisher Information entries

Here is the optimisation results for Pints and SciPy optimisers.

In [ ]:
print("Estimated parameters x1:", results.x)
print("Estimated parameters x2:", results2.x)

Estimated parameters x1: [0.00099888]
Estimated parameters x2: [0.001]
